In [9]:
# Import libaries
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import creds
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from recommender_v2 import cosine_recs


In [10]:
df = pd.read_csv('drake_songs_dataset.csv')
drake_sad = pd.read_csv('playlist_1.csv')
drake_hype = pd.read_csv('playlist_2.csv')
drake_chill = pd.read_csv('playlist_3.csv')
drake_romantic = pd.read_csv('playlist_4.csv')
drake_party = pd.read_csv('playlist_5.csv')

In [11]:
features = [
        'danceability', 'energy', 'key', 'loudness',
        'speechiness', 'acousticness', 'instrumentalness', 
        'liveness', 'valence', 'tempo'
    ]

In [12]:
def test_recs(sample_df, drake_df, features, model):
    # Randomly sample songs that will be input songs
    
    sample = sample_df.sample(n = 5, replace = False, random_state = 42)

    # Create output songs 
    recs = pd.DataFrame(columns = features)

    # Iterate through input songs and generate predictions
    for track in sample.iterrows():
        recs = pd.concat([recs, model(drake_df, features, 5, track[1]['track_name'])])

    # Get relevant columns and scale sample
    sample_features = sample[features].copy()
    scaler = StandardScaler()
    sample_scaled = scaler.fit_transform(sample_features)

    # Get relevant columsn and scale recs
    recs_features = recs[features].copy()
    recs_scaled = scaler.fit_transform(recs_features)

    # Calculate cosine similiarity between input songs
    input_input_matrix = cosine_similarity(sample_scaled, sample_scaled)

    # Calculate cosine similarity between output songs
    output_output_matrix = cosine_similarity(recs_scaled, recs_scaled)

    # Calculate cosine similarity matrix between input and output songs
    input_output_matrix = cosine_similarity(sample_scaled, recs_scaled)
    
    print(f"Intra-set input mean cosine similarity score: {input_input_matrix.mean()}")
    print(f"Intra-set output mean cosine similarity score: {output_output_matrix.mean()}")
    print(f"Inter-set mean cosine similarity score: {input_output_matrix.mean()}")
    print()


In [13]:
#mood_list = [drake_sad, drake_hype, drake_chill, drake_romantic, drake_party]

#for mood in mood_list:
    #test_recs(mood, df, features, cosine_recs)


In [24]:
def test_recs_v2(sample_df, drake_df, features, model):
    # Randomly sample songs that will be input songs
    
    sample = sample_df.sample(n = 5, replace = False, random_state = 42)

    # Get relevant columns and scale sample
    sample_features = sample[features].copy()
    scaler = StandardScaler()
    sample_scaled = scaler.fit_transform(sample_features)
    
    # List to store average similarity between recommendations and their input song
    avg_sim_scores = []
    # Iterate through input songs and generate predictions
    for i, track in enumerate(sample.iterrows()):
        # Generate recommendations based on the ith sampled track
        recs = model(drake_df, features, 5, track[1]['track_name'])
        # Get relevant columsn and scale recs
        recs_features = recs[features].copy()
        recs_scaled = scaler.fit_transform(recs_features)
        # Calculate cosine similarity matrix between input song and recs
        cos_matrix = cosine_similarity(sample_scaled[i].reshape(1, -1), recs_scaled)
        avg_sim_scores.append(cos_matrix.mean())
    return sum(avg_sim_scores) / len(avg_sim_scores)


    

In [25]:
test_recs_v2(drake_sad, df, features, cosine_recs)

0.0019364386774308